<a href="https://colab.research.google.com/github/iwa103/MintPy/blob/main/1_Geotiff%E7%B5%90%E5%90%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rasterio
!pip install h5py


## 変位用（複数のGeotiffあり）

ファイル読込は２回実施（１回目　抽出したいポイントの座標値（緯度経度）、２回目　グラフに取込みたい変位データのGeotiffファイルを選択（基本は全選択）

In [ ]:
# 変位用
import csv
import re
from google.colab import files
import rasterio

# 座標が記載されたテキストファイルのアップロード
uploaded_coord_file = files.upload()
coord_file_name = next(iter(uploaded_coord_file))

# 座標ファイルの読み込み
coordinates = []
with open(coord_file_name, 'r') as coord_file:
    csv_reader = csv.reader(coord_file)
    next(csv_reader)  # 最初の行をスキップする
    for line in csv_reader:
        point, lon, lat = line
        coordinates.append((point.strip(), float(lat.strip()), float(lon.strip())))

print(coordinates)

# GeoTIFFファイルのアップロード
uploaded_tiff_files = files.upload()

# 最初の行のヘッダー名を設定（例: "Point_A_value"）
first_point_header = f'{coordinates[0][0]}_value'

# CSVファイルのヘッダー（2つ目のポイントから始める）
csv_header = ['Date', first_point_header] + [f'{point}_value' for point, _, _ in coordinates[1:]]

# CSVファイルのファイル名
csv_file_name = 'geoTIFF_values.csv'

# 結果をCSVファイルに保存
with open(csv_file_name, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(csv_header)  # ヘッダーの書き込み

    for file_name in uploaded_tiff_files.keys():
        row = [re.search(r'\d{8}_\d{8}', file_name).group()]  # 日付の抽出
        for point, latitude, longitude in coordinates:
            with rasterio.open(file_name) as src:
                # 緯度経度をピクセル座標に変換
                r, c = src.index(longitude, latitude)
                # 範囲を超えていないか確認
                r = max(0, min(r, src.height - 1))
                c = max(0, min(c, src.width - 1))
                # 対応するピクセルの値を取得
                value = src.read(1)[r, c]
                row.append(value)
        csv_writer.writerow(row)

# CSVファイルのダウンロード
files.download(csv_file_name)


## 補助的なもの

## ファイルサイズが大きい場合用

In [ ]:
import csv
import re
import os
import rasterio

# 座標ファイルのパス
coord_file_path = '/content/高松グラフ用ポイント.csv'  # ここを座標ファイルのパスに変更

# 座標ファイルの読み込み
coordinates = []
with open(coord_file_path, 'r') as coord_file:
    csv_reader = csv.reader(coord_file)
    next(csv_reader)  # 最初の行をスキップする
    for line in csv_reader:
        point, lon, lat = line
        coordinates.append((point.strip(), float(lat.strip()), float(lon.strip())))

print(coordinates)

# GeoTIFFファイルが保存されているフォルダのパス
tiff_folder_path = '/content/data'  # ここをフォルダのパスに変更

# 最初の行のヘッダー名を設定（例: "Point_A_value"）
first_point_header = f'{coordinates[0][0]}_value'

# CSVファイルのヘッダー（2つ目のポイントから始める）
csv_header = ['Date', first_point_header] + [f'{point}_value' for point, _, _ in coordinates[1:]]

# CSVファイルのファイル名
csv_file_name = 'geoTIFF_values.csv'

# 結果をCSVファイルに保存
with open(csv_file_name, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(csv_header)  # ヘッダーの書き込み

    for file_name in os.listdir(tiff_folder_path):
        if file_name.endswith('.tif') or file_name.endswith('.tiff'):
            row = [re.search(r'\d{8}_\d{8}', file_name).group()]  # 日付の抽出
            full_file_path = os.path.join(tiff_folder_path, file_name)
            for point, latitude, longitude in coordinates:
                with rasterio.open(full_file_path) as src:
                    # 緯度経度をピクセル座標に変換
                    r, c = src.index(longitude, latitude)
                    # 範囲を超えていないか確認
                    r = max(0, min(r, src.height - 1))
                    c = max(0, min(c, src.width - 1))
                    # 対応するピクセルの値を取得
                    value = src.read(1)[r, c]
                    row.append(value)
            csv_writer.writerow(row)


## コヒーレンス用

In [ ]:
# 変位用
import csv
import re
from google.colab import files
import rasterio

# 座標が記載されたテキストファイルのアップロード
uploaded_coord_file = files.upload()
coord_file_name = next(iter(uploaded_coord_file))

# 座標ファイルの読み込み
coordinates = []
with open(coord_file_name, 'r') as coord_file:
    csv_reader = csv.reader(coord_file)
    next(csv_reader)  # 最初の行をスキップする
    for line in csv_reader:
        point, lon, lat = line
        coordinates.append((point.strip(), float(lat.strip()), float(lon.strip())))

print(coordinates)

# GeoTIFFファイルのアップロード
uploaded_tiff_files = files.upload()

# 最初の行のヘッダー名を設定（例: "Point_A_value"）
first_point_header = f'{coordinates[0][0]}_value'

# CSVファイルのヘッダー（2つ目のポイントから始める）
csv_header = ['Date', first_point_header] + [f'{point}_value' for point, _, _ in coordinates[1:]]

# CSVファイルのファイル名
csv_file_name = 'geoTIFF_values.csv'

# 結果をCSVファイルに保存
with open(csv_file_name, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(csv_header)  # ヘッダーの書き込み

    for file_name in uploaded_tiff_files.keys():
        # row = [re.search(r'\d{8}_\d{8}', file_name).group()]  # 日付の抽出 使わない！！
        row = [file_name]  # ファイル名を直接使用
        for point, latitude, longitude in coordinates:
            with rasterio.open(file_name) as src:
                # 緯度経度をピクセル座標に変換
                r, c = src.index(longitude, latitude)
                # 範囲を超えていないか確認
                r = max(0, min(r, src.height - 1))
                c = max(0, min(c, src.width - 1))
                # 対応するピクセルの値を取得
                value = src.read(1)[r, c]
                row.append(value)
        csv_writer.writerow(row)

# CSVファイルのダウンロード
files.download(csv_file_name)


## 水面や変位計算不可のデータ削除

In [ ]:
import pandas as pd

# CSVファイルを読み込む（ここではファイルパスを 'your_file.csv' としています）
file_path = '/content/geoTIFF_values.csv'
df = pd.read_csv(file_path)

# 3行目に値がない列を削除
df_filtered_columns = df.loc[:, df.iloc[2] != 0]

# 結果を表示
print(df_filtered_columns.head())

# 結果を新しいCSVファイルに保存する場合
df_filtered_columns.to_csv('filtered_data.csv', index=False)


## フィルタで残ったポイントからポイントファイル作成

In [ ]:
import pandas as pd

# CSVファイルを読み込む（ファイルパスは 'your_file.csv' に置き換えてください）
df = pd.read_csv('/content/filtered_data.csv')

# ヘッダー（最初の行）の2列目からの値を取得
header_values = df.columns[1:]  # 1列目を除外

# それぞれのヘッダー値を改行で区切ってCSV形式の文字列として保存
header_csv_string = "\n".join(header_values)

# CSV形式の文字列をファイルに書き出す
with open('header_values.csv', 'w') as file:
    file.write(header_csv_string)
